In [1]:
import pandas as pd
from plotly import graph_objects as go
import nevergrad as ng
from summer2 import AgeStratification, Overwrite

from estival.wrappers.nevergrad import optimize_model
import estival.priors as esp
import estival.targets as est
from estival.model import BayesianCompartmentalModel
from estival.wrappers import pymc as epm
from summer2 import CompartmentalModel
from summer2.parameters import Parameter, Function, DerivedOutput
from summer2.functions.time import get_sigmoidal_interpolation_function

from tb_incubator.demographics import add_extra_crude_birth_flow
from tb_incubator.constants import set_project_base_path
from tb_incubator.input import get_birth_rate, get_pop_death_data, get_death_rates

pd.options.plotting.backend = "plotly"
project_paths = set_project_base_path("../tb_incubator/")

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
# Load birth data
birth_rates = get_birth_rate()

/Users/tys/Documents/research/2024/TB Model/incubator2024/tb_incubator/input.py:13: DtypeWarning: Columns (2,3,4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  demographics = pd.read_csv(data_path / 'un_demographics.csv')


In [3]:
# Load age-stratified, population and death data
pop_death = get_pop_death_data()
target_pops = pop_death.groupby(level=[0]).sum()["population"]

/Users/tys/Documents/research/2024/TB Model/incubator2024/tb_incubator/input.py:32: DtypeWarning: Columns (3,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111) have mixed types. Specify dtype option on import or set low_memory=False.
  un_deaths = pd.read_csv(data_path / 'un_deaths_single_age.csv')
/Users/tys/Documents/research/2024/TB Model/incubator2024/tb_incubator/input.py:68: DtypeWarning: Columns (3,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111) have mixed

In [4]:
# Load death rates
death_rates = get_death_rates()

/Users/tys/Documents/research/2024/TB Model/incubator2024/tb_incubator/input.py:32: DtypeWarning: Columns (3,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111) have mixed types. Specify dtype option on import or set low_memory=False.
  un_deaths = pd.read_csv(data_path / 'un_deaths_single_age.csv')
/Users/tys/Documents/research/2024/TB Model/incubator2024/tb_incubator/input.py:68: DtypeWarning: Columns (3,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111) have mixed

In [5]:
# Arbitrary base model construction
model_comps = ["susceptible", "early latent", "late latent", "infectious", "recovered"]
model_times = [1850.0, 2024.0]
model = CompartmentalModel(
    times=model_times,
    compartments=model_comps,
    infectious_compartments=["infectious"],
)
model.set_initial_population({})

In [6]:
# TB transitions, some meaningless TB-related flows - can add to these to test
model.add_death_flow("TB death", Parameter("death rate"), "infectious")

In [7]:
# Demographic transitions
model.add_universal_death_flows("population_death", 1.0)  # Placeholder to overwrite later
model.add_replacement_birth_flow("replacement_birth", "susceptible")

In [8]:
# Population by year and get the duration of the run-in period
total_pop_by_year = pop_death.groupby("year")["population"].sum()
pop_start_year = total_pop_by_year.index[0]
start_period = pop_start_year - model_times[0]

In [15]:
pop_start_year

1950

In [9]:
# Calculate population entry rates and convert to function
pop_entry = total_pop_by_year.diff().dropna()  # Note this will only work if data are annual
pop_entry[pop_start_year] = total_pop_by_year[pop_start_year] / start_period
pop_entry = pop_entry.sort_index()
entry_rate = get_sigmoidal_interpolation_function(pop_entry.index, pop_entry)

In [10]:
# Apply age stratification with age-specific death rate functions of time
agegroup_request = [[0, 4], [5, 14], [15, 34], [35, 49], [50, 100]]
age_strata = [i[0] for i in agegroup_request]
strat = AgeStratification("age", age_strata, model_comps)
death_adjs = {}
for age in age_strata:
    years = death_rates.index
    rates = death_rates[age]
    pop_death_func = get_sigmoidal_interpolation_function(years, rates)
    death_adjs[str(age)] = Overwrite(pop_death_func)
strat.set_flow_adjustments("population_death", death_adjs)
model.stratify_with(strat)

In [11]:
# Add births as additional entry rate (split imports in case the susceptible compartments are further stratified later)
model.add_importation_flow("births", entry_rate, dest="susceptible", split_imports=True, dest_strata={"age": "0"})

In [12]:
# Track populations
age_pop_outputs = [model.request_output_for_compartments(s, model_comps, strata={"age": str(s)}) for s in age_strata]

In [13]:
# Run and inspect results
model.run({"death rate": 0.01})
fig = model.get_derived_outputs_df().plot.area()
fig.add_trace(go.Scatter(x=target_pops.index, y=target_pops, name="target", mode="markers", marker=dict(color="black", size=2.0)))